# Zadanie domowe -- interpolacja dwusześcienna

Interpolacja dwusześcienna, to podobnie jak w przypadku interpolacji dwuliniowej, rozszerzenie idei interpolacji jednowymiarowej na dwuwymiarową siatkę.
W trakcie jej obliczania wykorzystywane jest 16 pikseli z otoczenia (dla dwuliniowej 4).
Skutkuje to zwykle lepszymi wynikami - obraz wyjściowy jest bardziej gładki i z mniejszą liczbą artefaktów.
Ceną jest znaczny wzrost złożoności obliczeniowej (zostało to zaobserwowane podczas ćwiczenia).

Interpolacja dana jest wzorem:
\begin{equation}
I(i,j) = \sum_{i=0}^{3} \sum_{j=0}^{3} a_{ij} x^i y^j
\end{equation}

Zadanie sprowadza się zatem do wyznaczenia 16 współczynników $a_{ij}$.
W tym celu wykorzystuje się, oprócz wartość w puntach $A$ (0,0), $B$ (1 0), $C$ (1,1), $D$ (0,1) (por. rysunek dotyczący interpolacji dwuliniowej), także pochodne cząstkowe $A_x$, $A_y$, $A_{xy}$.
Pozwala to rozwiązać układ 16-tu równań.

Jeśli zgrupujemy parametry $a_{ij}$:
\begin{equation}
a = [ a_{00}~a_{10}~a_{20}~a_{30}~a_{01}~a_{11}~a_{21}~a_{31}~a_{02}~a_{12}~a_{22}~a_{32}~a_{03}~a_{13}~a_{23}~a_{33}]
\end{equation}

i przyjmiemy:
\begin{equation}
x = [A~B~D~C~A_x~B_x~D_x~C_x~A_y~B_y~D_y~C_y~A_{xy}~B_{xy}~D_{xy}~C_{xy}]^T
\end{equation}

To zagadnienie można opisać w postaci równania liniowego:
\begin{equation}
Aa = x
\end{equation}
gdzie macierz $A^{-1}$ dana jest wzorem:

\begin{equation}
A^{-1} =
\begin{bmatrix}
1& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0& 0 \\
0&  0&  0&  0&  1&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0 \\
-3&  3&  0&  0& -2& -1&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0 \\
2& -2&  0&  0&  1&  1&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0 \\
0&  0&  0&  0&  0&  0&  0&  0&  1&  0&  0&  0&  0&  0&  0&  0 \\
0&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0&  0&  1&  0&  0&  0 \\
0&  0&  0&  0&  0&  0&  0&  0& -3&  3&  0&  0& -2& -1&  0&  0 \\
0&  0&  0&  0&  0&  0&  0&  0&  2& -2&  0&  0&  1&  1&  0&  0 \\
-3&  0&  3&  0&  0&  0&  0&  0& -2&  0& -1&  0&  0&  0&  0&  0 \\
0&  0&  0&  0& -3&  0&  3&  0&  0&  0&  0&  0& -2&  0& -1&  0 \\
9& -9& -9&  9&  6&  3& -6& -3&  6& -6&  3& -3&  4&  2&  2&  1 \\
-6&  6&  6& -6& -3& -3&  3&  3& -4&  4& -2&  2& -2& -2& -1& -1 \\
2&  0& -2&  0&  0&  0&  0&  0&  1&  0&  1&  0&  0&  0&  0&  0 \\
0&  0&  0&  0&  2&  0& -2&  0&  0&  0&  0&  0&  1&  0&  1&  0 \\
-6&  6&  6& -6& -4& -2&  4&  2& -3&  3& -3&  3& -2& -1& -2& -1 \\
4& -4& -4&  4&  2&  2& -2& -2&  2& -2&  2& -2&  1&  1&  1&  1 \\
\end{bmatrix}
\end{equation}

Potrzebne w rozważaniach pochodne cząstkowe obliczane są wg. następującego przybliżenia (przykład dla punktu A):
$$A_x = \frac{I(i+1,j) - I(i-1,j)}{2}$$
$$A_y = \frac{I(i,j+1) - I(i,j-1)}{2}$$
$$A_{xy} = \frac{I(i+1,j+1) - I(i-1,j) - I(i,j-1) + I(i,j)}{4}$$

## Zadanie

Wykorzystując podane informacje zaimplementuj interpolację dwusześcienną.
Uwagi:
- macierz $A^{-1}$ dostępna jest w pliku *a_invert.py*
- trzeba się zastanowić nad potencjalnym wykraczaniem poza zakres obrazka (jak zwykle).

Ponadto dokonaj porównania liczby operacji arytmetycznych i dostępów do pamięci koniecznych przy realizacji obu metod interpolacji: dwuliniowej i dwusześciennej.

In [ ]:
import cv2
import os
import requests
from matplotlib import pyplot as plt
import numpy as np

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/05_Resolution/'

fileName = "ainvert.py"
if not os.path.exists(fileName):
    r = requests.get(url + fileName, allow_redirects=True)
    open(fileName, 'wb').write(r.content)

In [ ]:
import numpy as np
from ainvert import A_invert

def bicubic_interpolation(img, scale_h, scale_w):
    height, width = img.shape
    new_height = int(height * scale_h)
    new_width = int(width * scale_w)

    new_img = np.zeros((new_height, new_width))

    for i in range(new_height):
        for j in range(new_width):
            i1 = int(i / scale_h)
            j1 = int(j / scale_w)
            i2 = min(i1 + 1, height - 1)
            j2 = min(j1 + 1, width - 1)
            i_ = i / scale_h - i1
            j_ = j / scale_w - j1
            
            A = img[i1, j1]
            B = img[i1, j2]
            C = img[i2, j1]
            D = img[i2, j2]
            
            def _x(_i_, _j_):
                if _i_+1 < height and _i_-1 >= 0:
                    return (img[_i_+1, _j_].astype('int32') - img[_i_-1, _j_].astype('int32')) // 2
                return 0
            
            def _y(_i_, _j_):
                if _j_+1 < width and _j_-1 >= 0:
                    return (img[_i_, _j_+1].astype('int32') - img[_i_, _j_-1].astype('int32')) // 2
                return 0
            
            def _xy(_i_, _j_):
                if _i_+1 < height and _i_-1 >= 0 and _j_+1 < width and _j_-1 >= 0:
                    return (img[_i_+1, _j_+1].astype('int32') - img[_i_-1, _j_].astype('int32') - img[_i_, _j_-1].astype('int32') + img[_i_, _j_]).astype('int32') // 4
                return 0
            
            A_x = _x(i1, j1)
            A_y = _y(i1, j1)
            A_xy = _xy(i1, j1)
            
            B_x = _x(i1, j2)
            B_y = _y(i1, j2)
            B_xy = _xy(i1, j2)
            
            C_x = _x(i2, j1)
            C_y = _y(i2, j1)
            C_xy = _xy(i2, j1)
            
            D_x = _x(i2, j2)
            D_y = _y(i2, j2)
            D_xy = _xy(i2, j2)
            
            x = np.array([A, B, C, D, A_x, B_x, C_x, D_x, A_y, B_y, C_y, D_y, A_xy, B_xy, C_xy, D_xy]).transpose()
            a = np.dot(A_invert, x).reshape(4, 4)
            
            for k in range(4):
                for l in range(4):
                    new_img[i, j] += a[k, l] * (i_ ** k) * (j_ ** l)
    
    return new_img


In [ ]:
params = [
    ['img/parrot.bmp', 1.5, 1.5],
    ['img/parrot.bmp', 2.5, 2.5],
    ['img/parrot.bmp', 2.5, 1.5],
    ['img/parrot.bmp', 0.5, 0.8],
    ['img/clock.bmp', 1.5, 2.0],
    ['img/chessboard.bmp', 2, 2]
]


for file, s_w, s_h in params:
    img = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
    img_resized = bicubic_interpolation(img, s_w, s_h)
    
    plt.figure(figsize=(img_resized.shape[0]/100, img_resized.shape[1]/100), dpi=200)
    plt.figure(figsize=(10, 10))
    plt.xticks([]), plt.yticks([])
    plt.imshow(img_resized, cmap ="gray")